## Imports

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import log_loss, precision_score, accuracy_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import pandas as pd
import unicodedata
import string
import re
import numpy as np
from nltk.util import ngrams


## Load in DataFrame

In [6]:
tweets = pd.read_csv('../data/final_tweets_df.csv', index_col = 0, engine = 'python', error_bad_lines = False)

Skipping line 322505: unexpected end of data


In [12]:
tweets.dropna(axis = 0, how = 'any', inplace = True)

In [14]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322502 entries, 0 to 352485
Data columns (total 10 columns):
customer_tweet_id        322502 non-null int64
day_tweeted              322502 non-null object
original_text            322502 non-null object
company_name             322502 non-null object
company_response_text    322502 non-null object
minutes_to_respond       322502 non-null float64
Reponse_Speed            322502 non-null object
customer_tweet_text      322502 non-null object
data_lemmatized          322502 non-null object
Cluster Topic            322502 non-null int64
dtypes: float64(1), int64(2), object(7)
memory usage: 27.1+ MB


In [36]:
def day_to_num(day):
    if day == 'Sunday':
        day = 0
    elif day == 'Monday':
        day = 1
    elif day == 'Tuesday':
        day = 2
    elif day == 'Wednesday':
        day = 3
    elif day == 'Thursday':
        day = 4
    elif day == 'Friday':
        day = 5
    else:
        day = 6
    return day

In [37]:
tweets.day_tweeted = tweets.day_tweeted.apply(lambda x: day_to_num(x))

In [52]:
tweets['Reponse_Speed'] = tweets['Reponse_Speed'].apply(lambda x: 1 if x == 'Fast' else 0)

## Logistic Regression

In [50]:
X = np.array(train_vecs)
y = np.array(df.Reponse_Speed)

kf = KFold(5, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        solver='lbfgs',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # Logistic Regression SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='binary'))

print(f'Logistic Regression f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

NameError: name 'train_vecs' is not defined